In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
from wordcloud import WordCloud
import seaborn as sns
import re
import os
import spacy
from nltk.corpus import stopwords
import string
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils.vis_utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import Embedding
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.layers.merge import concatenate
%matplotlib inline

Using TensorFlow backend.
C:\Users\Sooryaprakash\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\Sooryaprakash\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\Sooryaprakash\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\Sooryaprakash\Anaconda3\lib\site-packages\tenso

In [4]:
# Pre-sets
nlp = spacy.load('en_core_web_sm')
PATH = 'D:\PGP AIML\Internship\SEO\Scraped Data\Enriched'
COLS = ['idx','Unnamed: 0', 'srchStr', 'rank', 'fullLink', 'respCD', 'title', 'h1',
       'h2', 'h3', 'h4', 'h5', 'h6', 'a', 'imgAlt', 'metaTitle', 'metaImage',
       'domain', 'uri', '%title', '%h1', '%h2', '%h3', '%h4', '%h5', '%h6',
       '%a', '%imgAlt', '%metaTitle', '%metaImage', '%domain', '%uri']

# Stop words
stop_words = set(stopwords.words('english'))
add_stopwords = ['//','java','script','html','nan','nav']
for wrds in add_stopwords:
    stop_words.add(wrds)

In [5]:
for ix,file in enumerate(os.listdir(PATH)):
    print(file)
    data = pd.read_excel(os.path.join(PATH,file))
    if ix > 0:
        tmp = np.vstack((tmp,data))  
    else:
        tmp = data

scraped_corona cases.xls
scraped_corona china.xls
scraped_corona deutschland.xls
scraped_corona in india.xls
scraped_corona india.xls
scraped_corona live.xls
scraped_corona news.xls
scraped_corona symptoms.xls
scraped_corona update.xls
scraped_corona usa.xls
scraped_corona virus update.xls
scraped_corona.xls
scraped_coronavirus.xls
scraped_india corona cases.xls
scraped_virus corona.xls
scraped_world corona.xls


In [6]:
FullExtract = pd.DataFrame(tmp,columns=COLS)

In [7]:
FullExtract.drop(columns=['idx','Unnamed: 0'],axis=1,inplace=True)

In [8]:
FullExtract.head()

,srchStr,rank,fullLink,respCD,title,h1,h2,h3,h4,h5,...,%h3,%h4,%h5,%h6,%a,%imgAlt,%metaTitle,%metaImage,%domain,%uri
0,corona cases,1,https://www.worldometers.info/coronavirus/,200,"Coronavirus Update (Live): 5,260,098 Cases and...","['Coronavirus Cases:', 'Deaths:', 'Recovered:']","['Latest News', 'How dangerous is the virus?\n...","['Reported Cases and Deaths by Country, Territ...","['May 22 (GMT)', ' Patient who died in the Phi...",NaN,...,0.5,0.5,0,0,1,0,0,0,0,0
1,corona cases,2,https://timesofindia.indiatimes.com/india/coro...,200,"Coronavirus in India live updates: 1,751 new ...",['Coronavirus live updates: 14-29 lakh Covid c...,"['Top Searches:', 'Trending Topics', 'Videos',...","['News Home', 'Editorials', 'Entertainment', '...",['India News'],NaN,...,0,0,0,0,1,0,0,0,0,1
2,corona cases,3,https://economictimes.indiatimes.com/news/poli...,200,Coronavirus Live Updates: Maharashtra records ...,['Coronavirus Live Updates: Maharashtra record...,"['Live Blog', 'Jammu & Kashmir: Mosques in Sri...","['Christ College of Engineering, Irinjalakuda,...","['subscribe to our newsletter', 'Become a memb...",['Have something to say? Post your comment'],...,0.5,0,0,0,1,0,0,0,0,1
3,corona cases,4,https://indianexpress.com/article/explained/co...,200,"India Coronavirus Cases Numbers Update: Delhi,...",['Coronavirus numbers explained: Bihar is beco...,['India Coronavirus (Covid-19) Cases Numbers e...,NaN,"['More Explained', 'Best of Express', 'Must Re...",NaN,...,0,0,0,0,1,0.5,0.5,0.5,0,0.5
4,corona cases,5,https://www.outlookindia.com/newsscroll/mp-cro...,200,"MP crosses 6,000 corona cases mark",NaN,"['MP crosses 6,000 corona cases mark', 'Nutrit...",[''],NaN,NaN,...,0,0,0,0,1,0,0,0,0,1


In [9]:
FullExtract.shape

(1487, 30)

In [10]:
#FullExtract.to_excel('Fullextract.xls')

In [11]:
FullExtractSucc = pd.DataFrame(FullExtract[FullExtract['respCD'] == 200])

In [12]:
FullExtractSucc.reset_index(inplace=True)

In [13]:
FullExtractSucc.drop(columns=['index'],inplace=True)

In [14]:
FullExtractSucc.head()

,srchStr,rank,fullLink,respCD,title,h1,h2,h3,h4,h5,...,%h3,%h4,%h5,%h6,%a,%imgAlt,%metaTitle,%metaImage,%domain,%uri
0,corona cases,1,https://www.worldometers.info/coronavirus/,200,"Coronavirus Update (Live): 5,260,098 Cases and...","['Coronavirus Cases:', 'Deaths:', 'Recovered:']","['Latest News', 'How dangerous is the virus?\n...","['Reported Cases and Deaths by Country, Territ...","['May 22 (GMT)', ' Patient who died in the Phi...",NaN,...,0.5,0.5,0,0,1,0,0,0,0,0
1,corona cases,2,https://timesofindia.indiatimes.com/india/coro...,200,"Coronavirus in India live updates: 1,751 new ...",['Coronavirus live updates: 14-29 lakh Covid c...,"['Top Searches:', 'Trending Topics', 'Videos',...","['News Home', 'Editorials', 'Entertainment', '...",['India News'],NaN,...,0,0,0,0,1,0,0,0,0,1
2,corona cases,3,https://economictimes.indiatimes.com/news/poli...,200,Coronavirus Live Updates: Maharashtra records ...,['Coronavirus Live Updates: Maharashtra record...,"['Live Blog', 'Jammu & Kashmir: Mosques in Sri...","['Christ College of Engineering, Irinjalakuda,...","['subscribe to our newsletter', 'Become a memb...",['Have something to say? Post your comment'],...,0.5,0,0,0,1,0,0,0,0,1
3,corona cases,4,https://indianexpress.com/article/explained/co...,200,"India Coronavirus Cases Numbers Update: Delhi,...",['Coronavirus numbers explained: Bihar is beco...,['India Coronavirus (Covid-19) Cases Numbers e...,NaN,"['More Explained', 'Best of Express', 'Must Re...",NaN,...,0,0,0,0,1,0.5,0.5,0.5,0,0.5
4,corona cases,5,https://www.outlookindia.com/newsscroll/mp-cro...,200,"MP crosses 6,000 corona cases mark",NaN,"['MP crosses 6,000 corona cases mark', 'Nutrit...",[''],NaN,NaN,...,0,0,0,0,1,0,0,0,0,1


In [15]:
# turn a doc into clean tokens
def clean_doc(doc):
    # split into tokens by white space
    tokens = doc.split()
    # remove punctuation from each token
    table = str.maketrans('', '', string.punctuation)
    tokens = [w.translate(table) for w in tokens]
    # remove remaining tokens that are not alphabetic
    tokens = [word for word in tokens if word.isalpha()]
    # remove numbers 
    tokens = [re.sub('[^A-Za-z_]', '', word) for word in tokens if word.isalpha()] 
    # filter out stop words
    tokens = [w for w in tokens if not w in stop_words]
    # filter out short tokens
    tokens = [word for word in tokens if len(word) > 1]
    #change all lower 
    tokens = [word.lower() for word in tokens] 
    return tokens

In [16]:
for colix in range(4,10):
    for ix,txt in FullExtractSucc.iterrows():
        if str(txt[colix]) == 'nan':
            txt[colix] = ''
        FullExtractSucc.iloc[ix][colix] = str(' '.join(clean_doc(txt[colix])))

In [17]:
FullExtractSucc.head()

,srchStr,rank,fullLink,respCD,title,h1,h2,h3,h4,h5,...,%h3,%h4,%h5,%h6,%a,%imgAlt,%metaTitle,%metaImage,%domain,%uri
0,corona cases,1,https://www.worldometers.info/coronavirus/,200,coronavirus update live cases deaths virus pan...,coronavirus cases deaths recovered,latest news how dangerous virusn how contagiou...,reported cases deaths country territory convey...,may gmt patient died philippines old male seri...,,...,0.5,0.5,0,0,1,0,0,0,0,0
1,corona cases,2,https://timesofindia.indiatimes.com/india/coro...,200,coronavirus india live updates new coronavirus...,coronavirus live updates lakh covid cases aver...,top searches trending topics videos related ne...,news home editorials entertainment videos tv b...,india news,,...,0,0,0,0,1,0,0,0,0,1
2,corona cases,3,https://economictimes.indiatimes.com/news/poli...,200,coronavirus live updates maharashtra records h...,coronavirus live updates maharashtra records h...,live blog jammu kashmir mosques srinagar remai...,christ college engineering irinjalakuda develo...,subscribe newsletter become member,have something say post comment,...,0.5,0,0,0,1,0,0,0,0,1
3,corona cases,4,https://indianexpress.com/article/explained/co...,200,india coronavirus cases numbers update delhi m...,coronavirus numbers explained bihar becoming t...,india coronavirus cases numbers explained most...,,more explained best express must read express ...,,...,0,0,0,0,1,0.5,0.5,0.5,0,0.5
4,corona cases,5,https://www.outlookindia.com/newsscroll/mp-cro...,200,mp crosses corona cases mark,,mp crosses corona cases mark nutrition tips to...,,,,...,0,0,0,0,1,0,0,0,0,1


In [22]:
def getWordCloud(words,title):
    # Create the wordcloud object
    wordcloud = WordCloud(width=480, height=480, margin=0).generate(words)
    # Display the generated image:
    plt.figure(figsize=(10,10))
    plt.title(title)
    plt.imshow(wordcloud, interpolation='bilinear')
    plt.axis("off")
    plt.margins(x=0, y=0)
    plt.show()

In [ ]:
# lets see the word cloud
label = ['title', 'h1','h2','h3','h4', 'h5']
for n in range(4,10):
    title = []
    Word = ''
    for ix,txt in FullExtractSucc.iterrows():
        Word+=txt[n] + ' '

    getWordCloud(titWord,label[n-5] + ' Word cloud')

In [24]:
# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

In [25]:
def max_length(col):
    return max([len(txt) for txt in FullExtractSucc[col]])

In [26]:
# calculate vocabulary size
def vocubSize(tokenizer):
    return len(tokenizer.word_index) + 1

In [27]:
# encode a list of lines
def encode_text(tokenizer, lines, length):
    # integer encode
    encoded = tokenizer.texts_to_sequences(lines)
    # pad encoded sequences
    padded = pad_sequences(encoded, maxlen=length, padding='post')
    return padded

In [28]:
def getTrain(col):
    tokenizer = create_tokenizer(FullExtractSucc[col].values)
    maxLen = max_length(col)
    vocSize = vocubSize(tokenizer)
    print(col + ' ' + str(vocSize))
    return encode_text(tokenizer,FullExtractSucc[col].values,maxLen)

In [29]:
label = ['title', 'h1','h2','h3','h4', 'h5']

In [30]:
X_title = getTrain('title')
X_h1 = getTrain('h1')
X_h2 = getTrain('h2')
X_h3 = getTrain('h3')
X_h4 = getTrain('h4')
X_h5 = getTrain('h5')

title 1822
h1 2617
h2 9249
h3 11067
h4 3852
h5 1440


In [31]:
y = FullExtractSucc['rank'].values

In [32]:
print(max_length('title'))
print(max_length('h1'))
print(max_length('h2'))
print(max_length('h3'))
print(max_length('h4'))
print(max_length('h5'))

222
1658
25114
23485
5535
2279


In [200]:
# Model creation 

In [34]:
# define the model
def define_model(vocab_size):
    # title
    inputs1 = Input(shape=(222,))
    embedding1 = Embedding(1822, 100)(inputs1)
    conv1 = Conv1D(filters=32, kernel_size=4, activation='relu')(embedding1)
    drop1 = Dropout(0.5)(conv1)
    pool1 = MaxPooling1D(pool_size=2)(drop1)
    conv11 = Conv1D(filters=32, kernel_size=4, activation='relu')(pool1)
    drop11 = Dropout(0.5)(conv11)
    pool11 = MaxPooling1D(pool_size=2)(drop11)
    flat1 = Flatten()(pool11)
    
    # h1
    inputs2 = Input(shape=(1658,))
    embedding2 = Embedding(2617, 100)(inputs2)
    conv2 = Conv1D(filters=32, kernel_size=2, activation='relu')(embedding2)
    drop2 = Dropout(0.5)(conv2)
    pool2 = MaxPooling1D(pool_size=2)(drop2)
    conv21 = Conv1D(filters=32, kernel_size=2, activation='relu')(pool2)
    drop21 = Dropout(0.5)(conv21)
    pool21 = MaxPooling1D(pool_size=2)(drop21)
    flat2 = Flatten()(pool21)
    
    # h2
    inputs3 = Input(shape=(25114,))
    embedding3 = Embedding(9249, 100)(inputs3)
    conv3 = Conv1D(filters=32, kernel_size=2, activation='relu')(embedding3)
    drop3 = Dropout(0.5)(conv3)
    pool3 = MaxPooling1D(pool_size=2)(drop3)
    conv31 = Conv1D(filters=32, kernel_size=2, activation='relu')(pool3)
    drop31 = Dropout(0.5)(conv31)
    pool31 = MaxPooling1D(pool_size=2)(drop31)
    flat3 = Flatten()(pool31)
    
    # h3
    inputs4 = Input(shape=(23485,))
    embedding4 = Embedding(11067, 100)(inputs4)
    conv4 = Conv1D(filters=52, kernel_size=2, activation='relu')(embedding4)
    drop4 = Dropout(0.5)(conv4)
    pool4 = MaxPooling1D(pool_size=2)(drop4)
    conv41 = Conv1D(filters=32, kernel_size=2, activation='relu')(pool4)
    drop41 = Dropout(0.5)(conv41)
    pool41 = MaxPooling1D(pool_size=2)(drop41)
    flat4 = Flatten()(pool41)
    
    # h4
    inputs5 = Input(shape=(5535,))
    embedding5 = Embedding(3852, 100)(inputs5)
    conv5 = Conv1D(filters=32, kernel_size=2, activation='relu')(embedding5)
    drop5 = Dropout(0.5)(conv5)
    pool5 = MaxPooling1D(pool_size=2)(drop5)
    conv51 = Conv1D(filters=32, kernel_size=2, activation='relu')(pool5)
    drop51 = Dropout(0.5)(conv51)
    pool51 = MaxPooling1D(pool_size=2)(drop51)
    flat5 = Flatten()(pool51)
    
    # h5
    inputs6 = Input(shape=(2279,))
    embedding6 = Embedding(1440, 100)(inputs6)
    conv6 = Conv1D(filters=32, kernel_size=2, activation='relu')(embedding6)
    drop6 = Dropout(0.5)(conv6)
    pool6 = MaxPooling1D(pool_size=2)(drop6)
    conv61 = Conv1D(filters=32, kernel_size=2, activation='relu')(pool6)
    drop61 = Dropout(0.5)(conv61)
    pool61 = MaxPooling1D(pool_size=2)(drop61)
    flat6 = Flatten()(pool61)

    
    merged = concatenate([flat1, flat2, flat3, flat4, flat5, flat6])
    
    # Dense layers
    dense1 = Dense(50, activation='relu')(merged)
    dense2 = Dense(50, activation='relu')(dense1)
    dense3 = Dense(50, activation='relu')(dense2)
    outputs = Dense(1, activation='sigmoid')(dense3)
    model = Model(inputs=[inputs1, inputs2, inputs3, inputs4, inputs5, inputs6], outputs=outputs)
    
    # compile
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    # summarize
    print(model.summary())
    
    #Plot model 
#     plot_model(model, show_shapes=True, to_file='multichannel.png')
    return model

In [35]:
model = define_model(1822)

Instructions for updating:
Colocations handled automatically by placer.
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 222)          0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            (None, 1658)         0                                            
__________________________________________________________________________________________________
input_3 (InputLayer)            (None, 25114)        0                                            
__________________________________________________________________________________________________
input_4 (InputLayer)            (None, 23485)        0                                            
____________________

dropout_6 (Dropout)             (None, 12555, 32)    0           conv1d_6[0][0]                   
__________________________________________________________________________________________________
dropout_8 (Dropout)             (None, 11741, 32)    0           conv1d_8[0][0]                   
__________________________________________________________________________________________________
dropout_10 (Dropout)            (None, 2766, 32)     0           conv1d_10[0][0]                  
__________________________________________________________________________________________________
dropout_12 (Dropout)            (None, 1138, 32)     0           conv1d_12[0][0]                  
__________________________________________________________________________________________________
max_pooling1d_2 (MaxPooling1D)  (None, 53, 32)       0           dropout_2[0][0]                  
__________________________________________________________________________________________________
max_poolin

In [ ]:
model.fit([X_title,X_h1,X_h2,X_h3,X_h4,X_h5], y, epochs=10, batch_size=16)

Instructions for updating:
Use tf.cast instead.
Epoch 1/10
 576/1386 [===========>..................] - ETA: 11:42 - loss: 0.4521 - accuracy: 0.062 - ETA: 8:16 - loss: -215.7496 - accuracy: 0.031 - ETA: 7:05 - loss: -400.9040 - accuracy: 0.020 - ETA: 6:26 - loss: -505.9362 - accuracy: 0.015 - ETA: 6:02 - loss: -559.5894 - accuracy: 0.012 - ETA: 5:44 - loss: -626.0805 - accuracy: 0.010 - ETA: 5:31 - loss: -629.5902 - accuracy: 0.008 - ETA: 5:20 - loss: -641.1901 - accuracy: 0.007 - ETA: 5:11 - loss: -651.2086 - accuracy: 0.006 - ETA: 5:02 - loss: -658.1274 - accuracy: 0.006 - ETA: 4:55 - loss: -676.6509 - accuracy: 0.005 - ETA: 4:50 - loss: -681.3758 - accuracy: 0.005 - ETA: 4:48 - loss: -688.0564 - accuracy: 0.004 - ETA: 4:42 - loss: -681.8970 - accuracy: 0.004 - ETA: 4:37 - loss: -690.1763 - accuracy: 0.004 - ETA: 4:33 - loss: -702.2782 - accuracy: 0.003 - ETA: 4:29 - loss: -710.9635 - accuracy: 0.003 - ETA: 4:29 - loss: -715.5285 - accuracy: 0.003 - ETA: 4:24 - loss: -726.5353 - accu

In [ ]:
# Save model
model.save('model.h5')